# 第11章 描述符descriptor

描述符是Python对象模型中的一种特殊协议，主要和4个魔术方法有关：`__get__ __set__ __delete__ __set_name__`。

任何一个实现了`__get__ __set__ __delete__`其中之一的类，都可以称为描述符类，它的实例则称为描述符对象。

要使用一个描述符，最常见的方法是把它的实例对象设置为其他类（常被称为owner类）的属性。

## 11.1 \_\_get\_\_

描述符的`__get__(self, instance, owner=None)`方法，会在访问`owner`类或`owner`类实例的对应属性时被触发，其方法的两个参数含义如下：

- instance：如果用类的实例来访问描述符属性，此参数为类的实例对象；如果通过类本身访问，该值为None。
- owner：描述符对象所绑定的类



In [32]:
class InfoDescriptor:
    """描述符类"""
    
    def __get__(self, instance, owner=None):
        print(f"描述符的__get__, instance: {str(instance.__class__.__name__)}, owner: {owner}")
        if not instance:
            print("没有实例的调用")
            return self
        
        # 值 10 既不存储在类字典中也不存储在实例字典中。相反，值 10 是在调用时才取到的。
        return 10
    

In [56]:
class MyABC:
    """使用描述符的类"""
    bar = InfoDescriptor()
    
    def __init__(self):
        self.x = 5


MyABC().bar
vars(MyABC())

描述符的__get__, instance: MyABC, owner: <class '__main__.MyABC'>


{'x': 5}

In [57]:
MyABC.bar

描述符的__get__, instance: NoneType, owner: <class '__main__.MyABC'>
没有实例的调用


从上述代码，我们可以看到：

因为描述符是绑定到类中，以获取属性实例化的形式存在的。所以`owner`总是能返回被绑定的类；

只有当通过被绑定的类的实例访问时，`instance` 才返回了值。


## 11.2 \_\_set\_\_

与`__get__`类似，`__set__`是在类修改绑定的描述符属性时，触发的行为。`__set__`方法后的参数含义如下：

- instance：属性当前绑定的实例对象
- value：待设置的属性值

与`__get__`方法不同，`__set__`仅对实例修改有效。对类修改描述符无效。

描述器的一种流行用法是托管对实例数据的访问。描述器被分配给类字典中的公开属性，而实际数据作为私有属性存储在实例字典中。当访问公开属性时，会触发描述器的 `__get__()` 和 `__set__()` 方法。

在下面的例子中，age 是公开属性，_age 是私有属性。当访问公开属性时，描述器会记录下查找或更新的日志：

In [58]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAgeAccess:

    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info('Accessing %r giving %r', 'age', value)
        return value

    def __set__(self, obj, value):
        logging.info('Updating %r to %r', 'age', value)
        obj._age = value

class Person:

    age = LoggedAgeAccess()             # Descriptor instance

    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1                   # Calls both __get__() and __set__()
        

In [72]:
mary = Person('Mary M', 30)
dave = Person('David D', 40)
mary.age
mary.birthday()
vars(mary)

INFO:root:Updating 'name' to 'Mary M'
INFO:root:Updating 'age' to 30
INFO:root:Updating 'name' to 'David D'
INFO:root:Updating 'age' to 40
INFO:root:Accessing 'age' giving 30
INFO:root:Accessing 'age' giving 30
INFO:root:Updating 'age' to 31


{'_name': 'Mary M', '_age': 31}

此示例的一个主要问题是私有名称 _age 在类 LoggedAgeAccess 中是硬耦合的。这意味着每个实例只能有一个用于记录的属性，并且其名称不可更改。

## 11.3 \_\_set_name\_\_

Python3.6 新增方法，它接受以下两个参数：
- owner：描述符类当前绑定的类
- name： 描述符所绑定的属性名称

__set_name__的触发时机是owner类被创建的时候。

在下面的例子中，Person 类具有两个描述器实例 name 和 age。当类 Person 被定义的时候，他回调了 LoggedAccess 中的 __set_name__() 来记录字段名称，让每个描述器拥有自己的 public_name 和 private_name：

In [64]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAccess:

    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        value = getattr(obj, self.private_name)
        logging.info('Accessing %r giving %r', self.public_name, value)
        return value

    def __set__(self, obj, value):
        logging.info('Updating %r to %r', self.public_name, value)
        setattr(obj, self.private_name, value)

class Person:

    name = LoggedAccess()                # First descriptor instance
    age = LoggedAccess()                 # Second descriptor instance

    def __init__(self, name, age):
        self.name = name                 # Calls the first descriptor
        self.age = age                   # Calls the second descriptor

    def birthday(self):
        self.age += 1

In [67]:
vars(vars(Person)['name'])

{'public_name': 'name', 'private_name': '_name'}

In [68]:
vars(vars(Person)['age'])

{'public_name': 'age', 'private_name': '_age'}

In [73]:
vars(Person)['name']

In [77]:
locals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'class InfoDescriptor:\n    """描述符类"""\n    \n    def __get__(self, instance, owner=None):\n        print(f"描述符的__get__, instance: {instance}, owner: {instance}")\n        if not instance：\n            print("没有实例的调用")\n            return self\n        return \'infomative descriptor\'\n    ',
  'class InfoDescriptor:\n    """描述符类"""\n    \n    def __get__(self, instance, owner=None):\n        print(f"描述符的__get__, instance: {instance}, owner: {instance}")\n        if not instance：\n            print("没有实例的调用")\n            return self\n        return \'infomative descriptor\'\n    ',
  'class InfoDescriptor:\n    """描述符类"""\n    \n    def __get__(self, instance, owner=None):\n        print(f"描述符的__get__, instance